# Objetivo:

Entregar a LLM as imagens para serem descritas, com o contexto do chunk onde se encontra

## Passo a Passo:

## 1 - Separar todos os chunks do texto
## 2 - Encontrar os chunks que contem imagens

In [ ]:
import os
import sys
import time

import regex as re

# Pega o diretório atual do notebook
notebook_dir = os.getcwd() # ou os.path.dirname(__file__) se fosse um script .py

# Assume que 'src' está no mesmo nível do notebook ou um nível acima
# Ajuste '..' conforme a estrutura do seu projeto
project_root = os.path.abspath(os.path.join(notebook_dir, '..')) # Volta um diretório

# Se o 'src' estiver diretamente no mesmo nível do notebook:
# project_root = notebook_dir

# Adiciona o diretório raiz do projeto ao sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from PIL import Image
import io
import base64

from src.classe_gemini import GeminiApiClient
from src.retrieve_img import retrieve_image_bytes_from_db
from src.interact_database_sql import save_image_description_to_db, get_image_description_from_db, non_described_images

import sqlite3

### 1 - Separar todos os chunks do texto

In [ ]:
# Regex para encontrar as linhas que começam com o padrão de data

diary_text_path = r"C:\Users\fuedj\Documents\Code\RAG_Dr_Voss_v2\drvossv2\data\dr_voss_md.md"

with open(diary_text_path, 'r', encoding='utf-8') as md:
    diary_text = md.read()

In [ ]:
date_pattern = r"(\d{1,2})(?:st|nd|rd|th)? Day of ([A-Za-z]+) (18\d{2}) - ([A-Za-z\s]+)"
image_pattern = r"!\[\]\((_page_\d+_Picture_\d+\.jpeg)\)"

matches = [(match, match.start()) for match in re.finditer(date_pattern, diary_text)]

all_days = []
for i, (match, start_pos) in enumerate(matches):
    # Extract date components and title
    day, month, year, title = match.groups()
    # Determine the end position (next match's start or end of text)
    end_pos = matches[i + 1][1] if i + 1 < len(matches) else len(diary_text)
    # Get the content including the date line
    content = diary_text[start_pos:end_pos].strip()
    
    # Extrair nomes das imagens, se houver
    image_names = [match.group(1) for match in re.finditer(image_pattern, content)]
    # Usar o primeiro nome de imagem encontrado ou uma string vazia se não houver
    image = image_names[0] if image_names else ""
    
    all_days.append({
        'day': day,
        'month': month,
        'year': year,
        'title': title.strip(),
        'content_raw': content,
        'content_without_image': content.replace(image, "").replace("[]()",""),
        'content_image_described': "",
        'image': image,
        'image_description': "",
        'chunks': []
    })

In [ ]:
len(all_days)

In [ ]:
all_days[:3]

### 2 - Encontrar os chunks que contém imagens

In [ ]:
# Filter chunks that contain the image 'jpeg' in their content
image_chunks = [
    chunk for chunk in all_days
    if chunk['image'] != ""
]

In [ ]:
image_chunks[:3]

In [ ]:
len(image_chunks)

Encontrar as imagens do DB

In [ ]:
id_image = non_described_images()
n_d_images = [item[1] for item in id_image]

In [ ]:
len(n_d_images)

In [ ]:
n_d_images[:5]

In [ ]:
image_chunks_to_process = []
for image in n_d_images:
    for chunk in image_chunks:
        if image in chunk['image']:
            image_chunks_to_process.append(chunk)

In [ ]:
len(image_chunks_to_process)

In [ ]:
image_chunks_to_process[-1]

In [ ]:
# --- Inicializa o cliente Gemini API ---
# Certifique-se de que a variável de ambiente 'GOOGLE_API_KEY' está definida com sua chave de API
try:
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("A variável de ambiente 'GOOGLE_API_KEY' não está definida.")
    
    gemini_client = GeminiApiClient(api_key=api_key)
except ValueError as e:
    print(f"Erro de configuração da API: {e}")
    exit() # Encerra o programa se a chave da API não estiver configurada


In [ ]:
image_chunks_to_process[-2:]

In [ ]:
# --- Busca dos bytes da imagem no DB ---
for _image in image_chunks_to_process:
    image_name = _image.get("image")
    image_bytes = retrieve_image_bytes_from_db(filename=image_name)

    if image_bytes:
        # --- Codifica os bytes da imagem para Base64 ---
        # A classe GeminiApiClient espera a imagem em formato base64
        base64_image = base64.b64encode(image_bytes).decode('utf-8')

        # --- Prepara os dados para a API Gemini (usando a estrutura esperada pela classe) ---
        image_part = {
            "inlineData": {
                "mimeType": "image/jpeg", # Confirme que este é o tipo MIME correto da sua imagem
                "data": base64_image
            }
        }
        
    # --- Chamada da API ---
    model_name = 'gemini-1.5-pro' # Ou 'gemini-1.5-pro' se preferir um modelo mais potente
    
    chunk_description = _image.get('content_without_image')
    
    prompt = f"""
    You are describing a image from a fantasy world in a travel log journey of Doctor Voss, a woman visiting the capital of Veridia.
    Here is the context of the image: {chunk_description}.
    
    Describe **only** the image. Bring **only the description**. Be concise - 300 characters max.
    If the doctor is on the image, describe her looks, skin color, appearence and others with great details.
    """
    
    prompt_parts = [
        {"text": f"{prompt}"},
        image_part
    ]

    print(f"Enviando imagem do banco de dados para a API Gemini (modelo: {model_name})...")

    # Chama o método da classe GeminiApiClient
    response_data = gemini_client.generate_multimodal_content(model_name, prompt_parts)

    print("\n--- Resposta Bruta da API Gemini ---")
    print(response_data) # Para ver a estrutura completa da resposta
    print("-----------------------------------")

    # Extrai o texto da resposta usando o método da classe
    time.sleep(3) # time for the response
    generated_text = gemini_client.extract_text_from_response(response_data)

    if generated_text:
        print("\n--- Texto Descritivo do Gemini ---")
        print(generated_text)
        print("---------------------------------")
    else:
        print("\nNão foi possível extrair texto da resposta do Gemini.")
        
    _ = save_image_description_to_db(filename=image_name, description=generated_text)


## Salvar descrições ao dicionário

In [ ]:
for day in all_days:
    if day['image'] != "":
        day['image_description'] = get_image_description_from_db(filename=day['image'])

In [ ]:
for day in all_days:
    if day['image'] != "":
        day['content_image_described'] = day['content_raw'].replace("![]("+day['image']+")", "Picture - " + day['image_description'])

In [141]:
all_days[:5]

[{'day': '1',
  'month': 'Frostfall',
  'year': '1855',
  'title': 'Arrival in the Capital of Veridia',
  'content_raw': "1st Day of Frostfall 1855 - Arrival in the Capital of Veridia\n\n![](_page_0_Picture_1.jpeg)\n\nToday marks my arrival in the capital city of Veridia, a place teeming with vibrant cultural heritage and an unyielding commitment to progress. Under Queen Isolde's famed patronage, the arts flourished here. Walking through the grand avenues, I was captivated by the array of sculptures and paintings, bearing testament to her legacy. My first stop was the regal Assembly House, where the Assembly of Voices, Veridia's main legislative body, convenes. The debate inside, I was told, revolved around initiatives to fulfill Veridia's ambitious goal of using 80% renewable energy by 2050. This pervasive focus on sustainability is palpable, infusing the city's very lifeblood.\n\nAs evening crept in, I joined locals indulging in Zelphar stew, a traditional Veridian dish that warms th

In [143]:
DB_PATH = r"C:\Users\fuedj\Documents\Code\RAG_Dr_Voss_v2\drvossv2\data\imagens.db"

def add_all_days_table(db_name: str = DB_PATH, days: list[dict] = None) -> None:
        # Conectar ao banco de dados SQLite (substitua 'database.db' pelo nome do seu banco)
    db_path = db_name  # Ajuste o caminho/nome do banco conforme necessário
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    if days is not None:
        # Criar a tabela all_days
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS all_days (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                day TEXT,
                month TEXT,
                year TEXT,
                title TEXT,
                content_raw TEXT,
                content_without_image TEXT,
                content_image_described TEXT,
                image TEXT,
                image_description TEXT
            )
        ''')

        # Inserir os dados na tabela all_days
        for day in days:
            cursor.execute('''
                INSERT INTO all_days (
                    day, month, year, title, content_raw,
                    content_without_image, content_image_described,
                    image, image_description
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                day['day'],
                day['month'],
                day['year'],
                day['title'],
                day['content_raw'],
                day['content_without_image'],
                day['content_image_described'],
                day['image'],
                day['image_description']
            ))

        # Confirmar as alterações e fechar a conexão
        conn.commit()
        conn.close()

        print("Dados inseridos na tabela 'all_days' com sucesso!")
    else:
        print("Invalid Input")
    

In [144]:
add_all_days_table(days=all_days)

Dados inseridos na tabela 'all_days' com sucesso!
